<a href="https://colab.research.google.com/github/980608/python/blob/main/TensorFlow_MNIST_training_and_freezing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow MNIST example

In [ ]:
import datetime
import tensorflow as tf
from tensorflow.keras import datasets as tfds

In [ ]:
(x_train, y_train), (x_test, y_test) = tfds.mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='input'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 7s 2ms/step - loss: 0.2948 - accuracy: 0.9141 - val_loss: 0.1339 - val_accuracy: 0.9600
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1449 - accuracy: 0.9571 - val_loss: 0.1015 - val_accuracy: 0.9700
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1069 - accuracy: 0.9680 - val_loss: 0.0852 - val_accuracy: 0.9744
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0902 - accuracy: 0.9721 - val_loss: 0.0823 - val_accuracy: 0.9751
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0729 - accuracy: 0.9773 - val_loss: 0.0791 - val_accuracy: 0.9760


In [ ]:
model.save('mnist_model')

INFO:tensorflow:Assets written to: mnist_model/assets


## Freezing the Model
https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb

In [ ]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
frozen_out_path = './'
frozen_graph_filename = 'frozen_graph'

full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers:")
for layer in layers:
  print(layer)

print("-" * 60)
print("Fronzen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)



------------------------------------------------------------
Frozen model layers:
x
sequential/input/Const
sequential/input/Reshape
sequential/dense/MatMul/ReadVariableOp/resource
sequential/dense/MatMul/ReadVariableOp
sequential/dense/MatMul
sequential/dense/BiasAdd/ReadVariableOp/resource
sequential/dense/BiasAdd/ReadVariableOp
sequential/dense/BiasAdd
sequential/dense/Relu
sequential/dropout/Identity
sequential/output/MatMul/ReadVariableOp/resource
sequential/output/MatMul/ReadVariableOp
sequential/output/MatMul
sequential/output/BiasAdd/ReadVariableOp/resource
sequential/output/BiasAdd/ReadVariableOp
sequential/output/BiasAdd
sequential/output/Softmax
Identity
------------------------------------------------------------
Fronzen model inputs: 
[<tf.Tensor 'x:0' shape=(None, 28, 28) dtype=float32>]
Frozen model outputs: 
[<tf.Tensor 'Identity:0' shape=(None, 10) dtype=float32>]


In [ ]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)


'./frozen_graph.pb'